In [ ]:
%cd ../..
%reload_ext autoreload
%autoreload 2

In [ ]:
import pyspiel
import math
import numpy as np
import matplotlib.pyplot as plt
import matplotlib
from tqdm import tqdm
from itertools import combinations

from alpha_one.model.model_manager import OpenSpielCheckpointManager, AlphaOneCheckpointManager, CFRCheckpointManager
from alpha_one.model.agent import IIGMCTSAgent, DMCTSAgent, OmniscientAgent, DirectInferenceAgent, RandomAgent, HybridAlphaOneDMCTSAgent, CFRAgent
from alpha_one.model.evaluation.agent import AgentEvaluator 
from alpha_one.metrics import AverageRewardRatingSystem, EloRatingSystem, TrueSkillRatingSystem
from alpha_one.train import MCTSConfig
from alpha_one.utils.state_to_value import state_to_value
from alpha_one.utils.mcts_II import initialize_bot_alphaone, ii_mcts_agent, IIGMCTSConfig
from alpha_one.utils.play import GameMachine
from alpha_one.utils.determinized_mcts import initialize_bot, compute_mcts_policy
from alpha_one.game.information_set import InformationSetGenerator
from alpha_one.plots import PlotManager

# 1. Game Setup

In [ ]:
game_name = "leduc_poker"
game = pyspiel.load_game(game_name)


# 2. Setup agents

In [ ]:
UCT_C = 5
max_mcts_simulations = 100

## 2.1. AlphaOne
LP-local-43 with checkpoint 15 seems to be good!

In [ ]:
run_name_alpha_one = "LP-local-43"

In [ ]:
n_previous_observations = 3
optimism = 0.1

In [ ]:
model_manager_alpha_one = AlphaOneCheckpointManager(game_name, run_name_alpha_one)

observation_model, game_model = model_manager_alpha_one.load_checkpoint(-1)
observation_model_untrained, game_model_untrained = model_manager_alpha_one.load_checkpoint(0)

In [ ]:
alphaone_mcts_config = IIGMCTSConfig(
    uct_c=UCT_C, 
    max_mcts_simulations=max_mcts_simulations,
    temperature=0,
    alpha_one=True, 
    state_to_value=state_to_value(game_name), 
    use_reward_polic=True, 
    n_previous_observations=n_previous_observations, 
    optimism=optimism)

alpha_one_agent = IIGMCTSAgent.from_config(game, observation_model, game_model, alphaone_mcts_config)
alpha_one_agent_untrained = IIGMCTSAgent.from_config(game, observation_model_untrained, game_model_untrained, alphaone_mcts_config)

## 2.2. D-MCTS

In [ ]:
run_name_d_mcts = "LP-local-1"

In [ ]:
model_manager_dmcts = OpenSpielCheckpointManager(game_name, run_name_d_mcts)

dmcts_model = model_manager_dmcts.load_checkpoint(-1)
dmcts_model_untrained = model_manager_dmcts.load_checkpoint(0)

In [ ]:
dmcts_mcts_config = MCTSConfig(uct_c=math.sqrt(2), 
                               max_mcts_simulations=100,
                               temperature=0,
                               determinized_MCTS=True, 
                               omniscient_observer=True,
                               use_reward_polic=True)

d_mcts_agent = DMCTSAgent(dmcts_model, dmcts_mcts_config)
d_mcts_agent_untrained = DMCTSAgent(dmcts_model_untrained, dmcts_mcts_config)
d_mcts_random_rollout_agent = DMCTSAgent(None, dmcts_mcts_config, n_rollouts=1)

## 2.3. Omniscient Agent

In [ ]:
run_name_omniscient = "LP-local-6"

In [ ]:
model_manager_omniscient = OpenSpielCheckpointManager(game_name, run_name_omniscient)

omniscient_model = model_manager_omniscient.load_checkpoint(-1)

In [ ]:
omniscient_mcts_config = MCTSConfig(UCT_C, max_mcts_simulations, 0, use_reward_policy=True, omniscient_observer=True)

omniscient_agent_untrained = OmniscientAgent(game, omniscient_mcts_config)
omniscient_agent = OmniscientAgent(game, omniscient_mcts_config, model=omniscient_model)

## 2.4. Blind Agent

In [ ]:
run_name_blind = "LP-12-blind-1"

In [ ]:
model_manager_blind = OpenSpielCheckpointManager(game_name, run_name_blind)

blind_model = model_manager_blind.load_checkpoint(-1)

In [ ]:
blind_agent = DirectInferenceAgent(blind_model)

## 2.5. Random Agent

In [ ]:
random_agent = RandomAgent(game)

## 2.6. Hybrid AlphaOne + D-MCTS (Requires 2.1. and 2.2.)

In [ ]:
hybrid_alpha_one_dmcts_agent = HybridAlphaOneDMCTSAgent(dmcts_model, observation_model, dmcts_mcts_config, state_to_value(game_name), n_previous_observations)

## 2.7. Hybrid AlphaOne + Omniscient (Requires 2.1. and 2.3.)

In [ ]:
hybrid_alpha_one_omniscient_agent = IIGMCTSAgent.from_config(game, observation_model, omniscient_model, alphaone_mcts_config)

## 2.8. Hybrid D-MCTS + Omniscient (Requires 2.2 and 2.3)

In [ ]:
hybrid_d_mcts_omniscient_agent = DMCTSAgent(omniscient_model, dmcts_mcts_config)

## 2.9. Super Hybrid (D-MCTS + Omniscient + AlphaOne)

In [ ]:
super_hybrid_agent = HybridAlphaOneDMCTSAgent(omniscient_model, observation_model, dmcts_mcts_config, state_to_value(game_name), n_previous_observations)

## 2.10. Counterfactual Regret Minimization

In [ ]:
run_name_cfr = 'LP-CFR-2'

In [ ]:
model_manager = CFRCheckpointManager(game_name, run_name_cfr)
cfr_policy_table = model_manager.load_checkpoint(-1)
cfr_agent = CFRAgent(cfr_policy_table, temperature=0)

# 3. Player Setup

Available Agents:
 - `alpha_one_agent`
 - `alpha_one_agent_untrained`
 - `d_mcts_agent`
 - `d_mcts_agent_untrained`
 - `d_mcts_random_rollout_agent`
 - `omniscient_agent`
 - `omniscient_agent_untrained`
 - `blind_agent`
 - `random_agent`
 - `hybrid_alpha_one_dmcts_agent`
 - `hybrid_alpha_one_omniscient_agent`
 - `hybrid_d_mcts_omniscient_agent`
 - `cfr_agent`

In [ ]:
player_setup = {
    1: d_mcts_agent,
    0: random_agent,
}

# 4.  Competition with 2 players

In [ ]:
game_machine = GameMachine(game)

In [ ]:
correct_guess = 0
incorrect_guess = 0
game_returns = []
for _ in tqdm(range(50)):
    game_machine.new_game()

    while not game_machine.is_finished():
        player = game_machine.current_player()
        agent = player_setup[player]
        
        if agent.is_information_set_agent():
            action, policy = agent.next_move(game_machine.get_information_set_generator())
        else:
            action, policy = agent.next_move(game_machine.get_state())
            
        #action = np.argmax(policy)
        if isinstance(agent, IIGMCTSAgent):
            guessed_state = agent.get_last_guessed_state()
            if guessed_state.__str__() == game_machine.state.__str__():
                correct_guess += 1
            else:
                incorrect_guess += 1
    
        game_machine.play_action(action)
            
    game_returns.append(game_machine.get_rewards())
            
            

In [ ]:
game_returns = np.array(game_returns)
average_return = game_returns.mean(axis=0)
print(f"Average return:")
print(f"---------------")
print(f"  {type(player_setup[0]).__name__}: {average_return[0]}")
print(f"  {type(player_setup[1]).__name__}: {average_return[1]}")
if correct_guess + incorrect_guess > 0:
    print(f" correct guess probability: {correct_guess/(correct_guess+incorrect_guess):0.2%}")

# 5. The Tournament of Tournaments

In [ ]:
agents = {
    alpha_one_agent: 'AlphaOne',
    alpha_one_agent_untrained: 'AlphaOne untrained',
    d_mcts_agent: 'D-MCTS',
    d_mcts_agent_untrained: 'D-MCTS untrained',
    d_mcts_random_rollout_agent: 'D-MCTS rollout',
    omniscient_agent: 'Omniscient',
    omniscient_agent_untrained: 'Omniscient untrained',
    blind_agent: 'Blind',
    random_agent: 'Random',
    hybrid_alpha_one_dmcts_agent: 'Hybrid AlphaOne + D-MCTS',
    hybrid_alpha_one_omniscient_agent: 'Hybrid AlphaOne + Omniscient',
    cfr_agent: 'CFR'
}
agents_by_key, agent_names_by_key = zip(*agents.items())

In [ ]:
evaluator = AgentEvaluator(game)
elo_rating_system = EloRatingSystem(40)
true_skill_rating_system = TrueSkillRatingSystem()
average_reward_rating_system = AverageRewardRatingSystem()
rating_systems = [elo_rating_system, true_skill_rating_system, average_reward_rating_system]

elo_ratings_history = []
true_skill_ratings_history = []
average_reward_ratings_history = []
ratings_histories = [elo_ratings_history, true_skill_ratings_history, average_reward_ratings_history]

rating_system_names = ['Elo Rating', 'TrueSkill Rating', 'Average Reward Rating']

In [ ]:
def play_match(agent_id_player_1, agent_id_player_2):
    match_outcome, trajectory = evaluator.evaluate(agents_by_key[agent_id_player_1], agents_by_key[agent_id_player_2])
    match_outcome.with_renamed_players({0: agent_id_player_1, 1: agent_id_player_2})
    return match_outcome

1 Matchday takes around 3 minutes

In [ ]:
n_match_days = 20
for _ in tqdm(range(n_match_days)):
    match_outcomes = []
    for agent_id_1, agent_id_2 in combinations(range(len(agents)), 2):
        match_outcomes.append(play_match(agent_id_1, agent_id_2))
        match_outcomes.append(play_match(agent_id_2, agent_id_1))
            
    for rating_system, ratings_history in zip(rating_systems, ratings_histories):
        rating_system.update_ratings(match_outcomes)
        ratings_history.append(rating_system.get_ratings())

In [ ]:
plot_manager = PlotManager.new_run(game_name, 'tournament')

# 6. Plot Results

In [ ]:
plot_manager = PlotManager.new_run(game_name, 'tournament')

In [ ]:
plot_manager.save(elo_rating_system.get_ratings(), 'elo_ratings')
plot_manager.save(true_skill_rating_system.get_ratings(), 'true_skill_ratings')
plot_manager.save(average_reward_rating_system.get_ratings(), 'average_reward_ratings')
#plot_manager.save(ratings_histories, 'ratings_histories')
plot_manager.save(agent_names_by_key, 'agent_names_by_key')

In [ ]:
colors = [0, 0, 1, 1, 1, 2, 2, 3, 4, 1, 0, 7]
line_style_config = {
    1: '-',
    2: '--',
    3: '-.',
    4: ':'
}
line_styles = [line_style_config[colors[:idx + 1].count(c)] for idx, c in enumerate(colors)]

In [ ]:
for rating_system, ratings_history, rating_system_name in zip(rating_systems, ratings_histories, rating_system_names):
    plt.figure(figsize=(12, 8))
    color_map = plt.cm.tab10(np.linspace(0,1,max(colors) + 1))
    plt.title(f"Tournament ({rating_system_name})")
    for player_id in range(len(agents)):
        plt.plot(range(1, len(ratings_history) + 1), [rating[player_id] for rating in ratings_history], label=agent_names_by_key[player_id], color=color_map[colors[player_id]], linestyle=line_styles[player_id])
    plt.legend()
    plt.xlabel("Matchday")
    plt.ylabel(rating_system_name)
    plot_manager.save_current_plot(f"tournament_{rating_system_name}.pdf")
    plt.show()

In [ ]:
print("Final average rewards:")
average_rewards = average_reward_rating_system.get_ratings()
average_rewards = sorted(average_rewards.items(), key=lambda x: x[1], reverse=True)
sorted_agents, _ = zip(*average_rewards)
for player_id in sorted_agents:
    print(f" - {agent_names_by_key[player_id]}: {average_reward_rating_system.get_rating(player_id):.2f}")

In [ ]:
win_probability_matrix = np.zeros((len(agents), len(agents)))
for agent_1 in range(len(agents)):
    for agent_2 in range(len(agents)):
        win_probability_matrix[agent_1, agent_2] = elo_rating_system.calculate_win_probability(elo_rating_system.get_rating(agent_1), elo_rating_system.get_rating(agent_2))

In [ ]:
plt.figure(figsize=(12, 8))
ax = plt.gca()
plt.title("Probabilities of winning against other models", pad=180, fontsize=20)
ax.matshow(win_probability_matrix, cmap=matplotlib.colors.LinearSegmentedColormap.from_list("", ["red", "white", "green"]))
ax.set_xticks(list(range(12)))
ax.set_yticks(list(range(12)))
ax.set_xticklabels(list(agent_names_by_key), rotation=90)
ax.set_yticklabels(list(agent_names_by_key))
ax.axis('image')
plt.tight_layout()
plot_manager.save_current_plot(f"tournament_winning_probabilities.pdf")
plt.show()

In [ ]:
average_reward_matrix = np.zeros((len(agents), len(agents)))
for agent_1 in range(len(agents)):
    for agent_2 in range(len(agents)):
        average_reward_matrix[agent_1, agent_2] = average_reward_rating_system.get_rating(agent_1) - average_reward_rating_system.get_rating(agent_2)

In [ ]:
plt.figure(figsize=(12, 8))
ax = plt.gca()
plt.title("Average rewards against other models", pad=180, fontsize=20)
ax.matshow(average_reward_matrix, cmap=matplotlib.colors.LinearSegmentedColormap.from_list("", ["red", "white", "green"]))
ax.set_xticks(list(range(12)))
ax.set_yticks(list(range(12)))
ax.set_xticklabels(list(agent_names_by_key), rotation=90)
ax.set_yticklabels(list(agent_names_by_key))
plt.tight_layout()
plot_manager.save_current_plot(f"tournament_average_rewards.pdf")
plt.show()